Exercises

In [3]:
import numpy as np

# 4.5.

In [4]:
def threshold_matrix(S, thresh, strategy='absolute', scale=False, penalty=0.0, binarize=False):
    """Treshold matrix in a relative fashion

    Notebook: C4/C4S2_SSM-Thresholding.ipynb

    Args:
        S (np.ndarray): Input matrix
        thresh (float or list): Treshold (meaning depends on strategy)
        strategy (str): Thresholding strategy ('absolute', 'relative', 'local') (Default value = 'absolute')
        scale (bool): If scale=True, then scaling of positive values to range [0,1] (Default value = False)
        penalty (float): Set values below treshold to value specified (Default value = 0.0)
        binarize (bool): Binarizes final matrix (positive: 1; otherwise: 0) (Default value = False)

    Returns:
        S_thresh (np.ndarray): Thresholded matrix
    """
    if np.min(S) < 0:
        raise Exception('All entries of the input matrix must be nonnegative')

    S_thresh = np.copy(S)
    N, M = S.shape
    num_cells = N * M

    if strategy == 'absolute':
        thresh_abs = thresh
        S_thresh[S_thresh < thresh] = 0

    if strategy == 'relative':
        thresh_rel = thresh
        num_cells_below_thresh = int(np.round(S_thresh.size*(1-thresh_rel)))
        if num_cells_below_thresh < num_cells:
            values_sorted = np.sort(S_thresh.flatten('F'))
            thresh_abs = values_sorted[num_cells_below_thresh]
            S_thresh[S_thresh < thresh_abs] = 0
        else:
            S_thresh = np.zeros([N, M])

    if strategy == 'local':
        thresh_rel_row = thresh[0]
        thresh_rel_col = thresh[1]
        S_binary_row = np.zeros([N, M])
        num_cells_row_below_thresh = int(np.round(M * (1-thresh_rel_row)))
        for n in range(N):
            row = S[n, :]
            values_sorted = np.sort(row)
            if num_cells_row_below_thresh < M:
                thresh_abs = values_sorted[num_cells_row_below_thresh]
                S_binary_row[n, :] = (row >= thresh_abs)
        S_binary_col = np.zeros([N, M])
        num_cells_col_below_thresh = int(np.round(N * (1-thresh_rel_col)))
        for m in range(M):
            col = S[:, m]
            values_sorted = np.sort(col)
            if num_cells_col_below_thresh < N:
                thresh_abs = values_sorted[num_cells_col_below_thresh]
                S_binary_col[:, m] = (col >= thresh_abs)
        S_thresh = S * S_binary_row * S_binary_col

    if scale:
        cell_val_zero = np.where(S_thresh == 0)
        cell_val_pos = np.where(S_thresh > 0)
        if len(cell_val_pos[0]) == 0:
            min_value = 0
        else:
            min_value = np.min(S_thresh[cell_val_pos])
        max_value = np.max(S_thresh)
        # print('min_value = ', min_value, ', max_value = ', max_value)
        if max_value > min_value:
            S_thresh = np.divide((S_thresh - min_value), (max_value - min_value))
            if len(cell_val_zero[0]) > 0:
                S_thresh[cell_val_zero] = penalty
        else:
            print('Condition max_value > min_value is voliated: output zero matrix')

    if binarize:
        S_thresh[S_thresh > 0] = 1
        S_thresh[S_thresh < 0] = 0
    return S_thresh

In [5]:
S = np.array([[1,1,2,2], [4,3,4,3], [1,1,2,2], [5,6,6,5]])
N, M = S.shape
print('Matrix S of dimension N=%d and M=%d:'%(N,M))
print(S)

Matrix S of dimension N=4 and M=4:
[[1 1 2 2]
 [4 3 4 3]
 [1 1 2 2]
 [5 6 6 5]]


In [7]:
# (a) Global thresholding using  𝜏=4 :
strategy = 'absolute'
thresh = 4
S_thresh = threshold_matrix(S, thresh=thresh, strategy=strategy, scale=0, penalty=0, binarize=0)
print(S_thresh)

[[0 0 0 0]
 [4 0 4 0]
 [0 0 0 0]
 [5 6 6 5]]


In [8]:
# (b) Global thresholding using  𝜏=4  as in (a) with subsequent linear scaling of the range  [𝜏,𝜇]  to  [0,1]  using  𝜇:=max{𝐒(𝑛,𝑚)∣𝑛,𝑚∈[1:4]} :
strategy = 'absolute'
thresh = 4
S_thresh = threshold_matrix(S, thresh=thresh, strategy=strategy, scale=1, penalty=0, binarize=0)
print(S_thresh)

[[0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 1.  1.  0.5]]


In [9]:
# (𝐜)  Global thresholding with subsequent linear scaling as in (b) and applying the penalty parameter  𝛿=−1 :
strategy = 'absolute'
thresh = 4
S_thresh = threshold_matrix(S, thresh=thresh, strategy=strategy, scale=1, penalty=-1, binarize=0)
print(S_thresh)

[[-1.  -1.  -1.  -1. ]
 [ 0.  -1.   0.  -1. ]
 [-1.  -1.  -1.  -1. ]
 [ 0.5  1.   1.   0.5]]


In [10]:
# (𝐝)  Relative thresholding using the relative threshold parameter  𝜌=0.5 :
strategy = 'relative'
thresh = 0.5
S_thresh = threshold_matrix(S, thresh=thresh, strategy=strategy, scale=0, penalty=0, binarize=0)
print(S_thresh)

[[0 0 0 0]
 [4 3 4 3]
 [0 0 0 0]
 [5 6 6 5]]


In [11]:
# (𝐞)  Local thresholding in a column- and rowwise fashion using  𝜌=0.5
strategy = 'local'
thresh = [0.5, 0.5]
S_thresh = threshold_matrix(S, thresh=thresh, strategy=strategy, scale=0, penalty=0, binarize=0)
print(S_thresh)

[[0. 0. 0. 0.]
 [4. 0. 4. 0.]
 [0. 0. 0. 0.]
 [0. 6. 6. 0.]]


In [ ]:
출처: 